In [2]:
#import
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

In [ ]:
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--no-sandbox')   # uncomment for diable sandbox
# chrome_options.add_argument('--headless')     # uncomment for make selenium run in background
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--window-size=1920,1080")

In [ ]:
# input

start_date = "20 Oct 2024"
end_date = "7 Nov 2024"
username = "username"  # input username
password = "password"  # input password

4

In [ ]:
folder = r'/Users/nachai/Downloads/selenium-downloaded-data'    # donwload folder
if not os.path.exists(folder):
    os.makedirs(folder)

prefs = {'download.default_directory' : folder}
chrome_options.add_experimental_option('prefs', prefs)

files = len(os.listdir(folder))
print(f"There're {files} files in {folder}")

In [ ]:
# start web driver
driver = webdriver.Chrome(options=chrome_options)

driver.get('https://merchant.grab.com/portal')  # grab login url
time.sleep(1)

In [24]:
try:
    username_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "username")),
        message="Cannot access login url - timeout"
    )
    username_input.send_keys(username)
    time.sleep(1)

except Exception as error:
    print("--- error",error)
    raise error

password_input = driver.find_element(by=By.ID, value='password')
password_input.send_keys(password)

enter = driver.find_element(by=By.XPATH, value=r'//*[@id="root"]//form//button[*[contains(text(), "Log In")]]')


In [ ]:
# click enter login

driver.execute_script("arguments[0].click()", enter)
driver.implicitly_wait(2)
time.sleep(1)

In [ ]:
# if another user's using.
try:
    reject_cookie = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="onetrust-reject-all-handler"]')),
    )
    driver.execute_script("arguments[0].click()", reject_cookie)

    try:
        modal_continue = r'/html/body//div[contains(@class, "modal-footer")]//button[*[contains(text(), "Continue")]]'
        modal_continue = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, modal_continue)),
            message="Cannot access login url - timeout"
        )
        print('-- found another user login')
        driver.execute_script("arguments[0].click()", modal_continue)
    except:
        pass   
    
except:
    pass

In [ ]:
# close suggestion
try:
    modal_close_xpath = r'/html/body//button[span[contains(text(), "Close")]]'
    modal_close = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, modal_close_xpath)),
        message="Cannot access login url - timeout"
    )
    driver.execute_script("arguments[0].click()", modal_close)
except:
    pass

In [28]:
# go to finance page
driver.get('https://merchant.grab.com/finances')
time.sleep(1)

In [ ]:
# fill date

# click date picker
try:
    toggle_date_picker_xpath = r'//*[@id="rc-tabs-0-panel-transactions"]//div[input[@name="mex-datepicker_range-picker"]]'
    date_picker = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, toggle_date_picker_xpath)),
        message="Cannot access login url - timeout"
    )
    driver.execute_script("arguments[0].click()", date_picker)
    driver.implicitly_wait(2)
except Exception as error:
    print('No date picker')
    raise error
time.sleep(1)


# fill start date
start_date_input_xpath = r'/html/body//div[contains(@class, "rc-calendar-date-input-wrap")]//input[@placeholder="start"]'
start_date_input = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, start_date_input_xpath)),
    message="Cannot find start date input"
)

text = start_date_input.get_attribute('value')
start_date_input.click()
driver.execute_script(f"arguments[0].value=''", start_date_input)
driver.implicitly_wait(1)
start_date_input.send_keys(start_date)
time.sleep(1)
print('set start date', start_date)

# fill end date
end_date_input_xpath = r'/html/body//div[contains(@class, "rc-calendar-date-input-wrap")]//input[@placeholder="end"]'
end_date_input = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, end_date_input_xpath)),
    message="Cannot find end date input"
)

end_date_input.click()
driver.execute_script(f"arguments[0].value=''", end_date_input)
driver.implicitly_wait(1)
end_date_input.send_keys(end_date)
time.sleep(1)
print('set end date', end_date)
    
driver.execute_script("arguments[0].click()", date_picker)
driver.implicitly_wait(1)
time.sleep(3)


---text 11 Nov 2024
---tag input
---start_date_input <selenium.webdriver.remote.webelement.WebElement (session="9dfbd23bbaebce5be40f838f22aa8a8d", element="f.E8CF06716714D760DC1AD665E1FE12E5.d.8A35FFA305559FE2E7ED1219439B70B3.e.121")>
set start date 20 Oct 2024
set end date 7 Nov 2024


In [ ]:
# helper functions
def _find_element(element, by: str, by_value: str):
    try:
        return element.find_element(by, by_value)
    except Exception as e:
        # Log.extract_log(f"Not found element: {e.msg}")
        return e
    
def _find_elements(element, by: str, by_value: str):
    try:
        return element.find_elements(by, by_value)
    except Exception as e:
        return e
    
def wait_fetch_selected_table(by: str, by_value: str):
    def _predicate(driver):
        select_element = _find_element(driver, by, by_value)
        if isinstance(select_element, Exception):
            return False
        option_elements = _find_elements(select_element, By.TAG_NAME, "tr")
        if isinstance(option_elements, Exception):
            return False

        return len(option_elements) > 1

    return _predicate


In [ ]:
# wait fetching data

try:
    table_body_xpath = r'//*[@id="rc-tabs-0-panel-transactions"]//table[@class="mex-table"]/tbody'
    table_body = WebDriverWait(driver, 15).until(
        wait_fetch_selected_table(By.XPATH, table_body_xpath),
        message="Cannot fetch transaction detail"
    )
    print("table_body", table_body)
except Exception as e:
    print("No table", table_body)

    raise e



---options  4
table_body True


In [32]:
# recheck selected date
try:
    input_date_picker_xpath = r'//*[@id="rc-tabs-0-panel-transactions"]//input[@name="mex-datepicker_range-picker"]'
    input_date_picker = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, input_date_picker_xpath)),
        message="Cannot access login url - timeout"
    )
    selected_date = input_date_picker.get_attribute("value")
    print("selected_date", selected_date)
    # driver.execute_script("arguments[0].click()", date_picker)
    # date_picker.send_keys("30 Oct 2024 - 6 Nov 2024")
except:
    pass

selected_date 20 Oct 2024 - 7 Nov 2024


In [33]:
# click download
time.sleep(1)
download_xpath = driver.find_element(By.XPATH, r'//*[@id="rc-tabs-0-panel-transactions"]//button[span/span[contains(text(), "Download")]]')
driver.execute_script("arguments[0].click()", download_xpath)

In [ ]:
# select all 
time.sleep(1)
try:
    input_radio_all_xpath = r'//input[@name="radio-download" and @value="allTransaction"]'
    radio_input = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, input_radio_all_xpath)),
        message="Cannot access login url - timeout"
    )
    
    radio_input.click()
    driver.execute_script("arguments[0].checked = true", radio_input)
    driver.implicitly_wait(1)
except:
    pass


In [35]:
# click download
time.sleep(1)
try:
    download_button_xpath = r'/html/body//div[@class="modal-footer"]//button[@data-testid="btnDownloadTrxDetail"]'
    download_button = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, download_button_xpath)),
        message="No download button - timeout"
    )
    driver.execute_script("arguments[0].click()", download_button)
    driver.implicitly_wait(2)
except:
    pass


In [ ]:
# wait download complete

def wait_file_download(timeout: int):
    timeout = 5
    end_time = time.monotonic() + timeout
    dl_wait = True
    while dl_wait and time.monotonic() <= end_time:
        time.sleep(1)
        dirs = os.listdir(folder)
        count_file = len(dirs)
        for filename in dirs:
            if filename.endswith('.crdownload'):
                print(' downloading', filename)
                dl_wait = True
                break
        else:
            dl_wait = False
            break

wait_file_download(10)  # wait download file for 10 seconds


In [ ]:
# logout
time.sleep(1)

try:
    sidebar_toggle_xpath = r'//*[@id="root"]//button[.//span[@aria-label="chevron-right"]]'
    sidebar_toggle = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, sidebar_toggle_xpath)),
        message="No download button - timeout"
    )
    driver.execute_script("arguments[0].click()", sidebar_toggle)
    driver.implicitly_wait(1)
except:
    pass

try:
    setting_xpath = r'//*[@id="root"]//button[.//span[@aria-label="settings"]]'
    setting = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, setting_xpath)),
        message="No download button - timeout"
    )
    driver.execute_script("arguments[0].click()", setting)
    driver.implicitly_wait(1)

except:
    pass


try:
    logout_button_xpath = r'//*[@id="root"]//div[*[contains(text(), "Log Out") and @data-testid="btnLogOut"]]'
    logout_button = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, logout_button_xpath)),
        message="No download button - timeout"
    )
    logout_button.click()
    driver.execute_script("arguments[0].click()", logout_button)
    driver.implicitly_wait(1)
except:
    pass

In [ ]:
# close and quit

driver.close()
driver.quit()